# Task Desription

Please read first the following abstract

## Abstract
                
RNA-Seq reveals an unprecedented complexity of the neuroblastoma transcriptome and is suitable for clinical endpoint prediction [ microarray ]

### Experiment Description  

We generated gene expression profiles from 498 primary neuroblastomas using RNA-Seq and microarrays. We sought to systematically evaluate the capability of RNA deep-sequencing (RNA-Seq)-based classification for clinical endpoint prediction in comparison to microarray-based ones. The neuroblastoma cohort was randomly divided into training and validation sets (**Please note:** <em>in the following we refer to this validation set as test set</em>), and 360 predictive models on six clinical endpoints were generated and evaluated. While prediction performances did not differ considerably between the two technical platforms, the RNA-Seq data processing pipelines, or feature levels (i.e., gene, transcript, and exon junction levels), RNA-Seq models based on the AceView database performed best on most endpoints. Collectively, our study reveals an unprecedented complexity of the neuroblastoma transcriptome, and provides guidelines for the development of gene expression-based predictive classifiers using high-throughput technologies.  Sample clinical characteristics definitions:  

* sex: 
    <ul>
    <li>M = male</li>
    <li>F = female</li>
    </ul>
    
* age at diagnosis: The age in days at diagnosis 
    <ul>
    <li>integer</li>
    </ul>

* high risk: Clinically considered as high-risk neuroblastoma
    <ul>
    <li>yes = 1</li>
    <li>no = 0</li>
    </ul>


* INSS stage: Disease stage according to International Neuroblastoma Staging System ([INSS](https://www.cancer.org/cancer/neuroblastoma/detection-diagnosis-staging/staging.html)) 
    <ul>
    <li>1</li>
    <li>2</li>
    <li>3</li>
    <li>4</li>
    <li>4S</li>
    </ul>


* progression: Occurrence of a tumor progression event
    <ul>
    <li>yes = 1</li>
    <li>no = 0</li>
    </ul>



* death from disease: Occurrence of death from the disease (yes=1; no=0) 
    <ul>
    <li>yes = 1</li>
    <li>no = 0</li>
    </ul>





Gene expression of 498 neuroblastoma samples was quantified by RNA sequencing as well as by microarray analyses in order to understand the neuroblastoma transcriptome and predict clinical endpoints. 


## Task

The task is to predict the missing values in the validation set (from here on called test set). Do this either with the RNASeq or the Microarray data, or potentially both?



## Code

To make your start a bit easier, here is a small notebook reading the data in. It finishes with a function enabling you to save your predictions for submission. 

#### from here, the code starts

First some imports 

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt # plotting and visulisation
import seaborn as sns # nicer (easier) visualisation
%matplotlib inline


# for saving
import os,os.path

### Setting  up directory and filenames

In [4]:
data_dir = '~/Documents/MSc/module 3/grp/data/'.format(os.path.sep)

fn_fpkm             = 'log2FPKM.tsv'
fn_patient_info     = 'patientInfo.tsv'
fn_prop_intensities = 'allProbIntensities.tsv'




### Load the microarray data

This part already sets the indeces in the DataFrame. Please feel free to change as required. 

In [5]:
df_fpkm = pd.read_csv('{}/{}'.format(data_dir,fn_fpkm),sep='\t',).rename({'00gene_id':'gene_id'},axis=1)
df_fpkm = df_fpkm.set_index(['gene_id'])
df_fpkm.columns.name = 'ID'

df_fpkm.head()

ID,NB001,NB002,NB003,NB004,NB005,NB006,NB007,NB008,NB009,NB010,...,NB489,NB490,NB491,NB492,NB493,NB494,NB495,NB496,NB497,NB498
gene_id,,,,,,,,,,,,,,,,,,,,,
1/2-SBSRNA4,0.834381,0.743094,0.909414,0.795775,0.905540,0.869154,1.811352,0.599240,0.981855,1.066399,...,0.997977,1.003559,0.842437,1.057873,0.805515,0.491331,0.868249,0.911379,0.660139,1.152988
A1BG,1.910053,0.941996,1.950857,1.989477,1.942946,1.927608,1.617745,2.161291,1.436439,2.159797,...,2.336929,2.836360,1.205317,2.439868,1.649027,1.451425,1.493852,1.641241,1.994978,1.289534
A1BG-AS1,1.453191,0.640614,1.156765,1.525277,1.365043,0.899212,1.304178,1.189205,0.771248,1.114787,...,1.182908,1.367371,0.643751,1.096815,0.925425,0.933275,1.208723,0.904511,1.529221,1.102866
A1CF,0.005102,0.005902,0.005192,0.000000,0.025347,0.005682,0.000000,0.000000,0.021880,0.000000,...,0.024298,0.007295,0.000000,0.006678,0.005746,0.004998,0.004853,0.000000,0.022780,0.018720
A2LD1,0.580151,0.738233,0.927667,0.936497,0.924853,0.739038,1.018705,0.546324,0.666877,0.865850,...,0.673627,1.401265,0.837443,0.939849,0.743496,0.957837,0.812093,0.488748,1.068072,0.782887


### Load the RNASeq data

This part already sets the indeces in the DataFrame. Please feel free to change as required. 

In [6]:
df_prop_intensities = pd.read_csv('{}/{}'.format(data_dir,fn_prop_intensities),sep='\t').set_index(['Reporter.Identifier'])
df_prop_intensities.columns.name = 'ID'

df_prop_intensities.head()


ID,GeneSymbols,NB001,NB002,NB003,NB004,NB005,NB006,NB007,NB008,NB009,...,NB489,NB490,NB491,NB492,NB493,NB494,NB495,NB496,NB497,NB498
Reporter.Identifier,,,,,,,,,,,,,,,,,,,,,
28913,NaN,14.99,14.94,12.48,14.63,11.89,15.09,13.07,12.00,11.70,...,13.62,13.03,14.98,13.36,13.90,13.00,13.79,14.70,14.03,12.31
27262,NaN,9.20,10.41,9.27,8.83,7.97,10.33,9.62,8.72,9.36,...,6.26,5.93,6.97,5.99,7.62,7.76,8.56,7.74,7.57,7.08
3180,NaN,5.06,5.26,6.45,2.89,2.00,4.80,3.05,6.39,6.43,...,0.93,0.58,1.26,1.38,3.49,2.07,2.26,2.29,2.63,2.54
41426,MBL1P,7.45,8.68,6.30,7.30,6.26,7.50,7.43,6.98,8.02,...,5.35,5.57,5.51,6.30,6.60,6.38,7.49,6.77,8.13,7.11
37033,NaN,6.74,6.63,6.75,6.20,6.57,6.01,6.78,4.80,5.15,...,4.58,4.61,3.54,4.55,4.20,7.16,7.07,5.07,6.28,6.34


In [7]:
gene_list = ["KIFB1B2","MYCN", "NF1", "LET7", "TERT", "PHOX2B", "ALK", "ARTX", "CD798", "SOX9", "RPTOR", "TP53", "PRPN11", "NRAS", "PIK3CA", "FGFR1", "BDNF", "ASCL1", "RET", "MSH2"]
genes_of_interest = df_prop_intensities.loc[df_prop_intensities["GeneSymbols"].isin(gene_list)]

In [47]:
genes_of_interest.head(20)

ID,GeneSymbols,NB001,NB002,NB003,NB004,NB005,NB006,NB007,NB008,NB009,...,NB489,NB490,NB491,NB492,NB493,NB494,NB495,NB496,NB497,NB498
Reporter.Identifier,,,,,,,,,,,,,,,,,,,,,
36700,TERT,9.28,9.76,9.33,9.29,8.89,9.39,9.12,8.30,9.43,...,9.04,7.95,10.07,8.59,8.44,8.72,9.10,8.84,8.40,8.50
35332,NF1,8.62,6.44,2.54,5.61,4.41,4.15,6.94,2.96,4.15,...,3.50,5.43,4.52,4.90,4.86,5.28,6.06,4.15,6.62,7.09
40594,BDNF,10.67,11.39,10.50,11.28,9.82,10.14,10.46,10.75,10.80,...,9.68,8.60,10.33,9.27,11.65,10.91,11.55,11.40,11.84,10.88
813,TP53,5.65,5.06,5.93,2.96,2.10,4.49,3.10,6.12,6.12,...,3.32,0.58,1.38,2.07,2.58,3.79,2.41,3.64,3.28,3.81
811,TP53,11.42,11.27,10.31,11.00,11.01,10.94,11.33,10.98,10.58,...,10.85,11.86,11.14,10.45,10.13,10.66,10.80,10.30,10.62,10.88
38444,TERT,4.67,8.90,6.43,5.72,5.21,7.69,7.72,9.12,8.21,...,7.35,8.49,11.68,5.61,11.59,5.42,5.31,6.49,6.18,5.27
44248,TERT,9.00,10.31,9.29,8.88,8.83,10.10,10.35,9.24,10.11,...,7.26,8.08,7.95,7.27,8.41,8.25,9.29,9.10,8.89,8.54
44494,RET,13.26,14.40,12.51,13.43,12.48,14.10,13.01,12.32,12.39,...,12.43,12.60,13.39,12.62,14.39,13.89,14.22,14.08,14.36,13.47
34963,MSH2,11.20,10.69,8.28,8.48,6.74,11.49,10.06,7.70,9.94,...,5.00,3.55,4.80,5.43,5.93,5.00,6.75,6.20,6.69,9.71


In [8]:
tp_gene_of_i = genes_of_interest.transpose()
tp_gene_of_i

Reporter.Identifier,36700,35332,40594,813,811,38444,44248,44494,34963,42490,36855,42910,34175,44328,36747,43552,38595
ID,,,,,,,,,,,,,,,,,
GeneSymbols,TERT,NF1,BDNF,TP53,TP53,TERT,TERT,RET,MSH2,TP53,PHOX2B,TP53,RET,TP53,RET,TERT,BDNF
NB001,9.28,8.62,10.67,5.65,11.42,4.67,9,13.26,11.2,11.65,11.13,7.14,9.21,9.12,12.21,11.99,10.04
NB002,9.76,6.44,11.39,5.06,11.27,8.9,10.31,14.4,10.69,11.92,11.14,7.82,9.28,7.89,12.73,12.63,10.08
NB003,9.33,2.54,10.5,5.93,10.31,6.43,9.29,12.51,8.28,11.31,10.41,6.96,7.86,8.73,12.82,12.1,9.39
NB004,9.29,5.61,11.28,2.96,11,5.72,8.88,13.43,8.48,11.64,10.78,7.49,8.11,8.86,12.76,13.23,9.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NB494,8.72,5.28,10.91,3.79,10.66,5.42,8.25,13.89,5,10.82,10.09,7.42,8.75,8.43,12.88,13.44,8.7
NB495,9.1,6.06,11.55,2.41,10.8,5.31,9.29,14.22,6.75,11.54,10.5,7.45,8.96,9.1,12.84,13.62,8.95
NB496,8.84,4.15,11.4,3.64,10.3,6.49,9.1,14.08,6.2,11.42,10.68,8.15,8.61,8.54,12.46,12.13,9.05


### Load the patient factors, including the potential endpoints 

This part already sets the indeces in the DataFrame. Please feel free to change as required. 
Please note, that the ```FactorValues``` should have a 1-to-1 correspondence to the factors desc ribed in the abstract. 

In [9]:
df_patient_info = pd.read_csv('{}/{}'.format(data_dir,fn_patient_info),sep='\t').set_index('ID')
df_patient_info.columns.name = 'FactorValues'

df_patient_info.head()

FactorValues,FactorValue..Sex.,FactorValue..age.at.diagnosis.,FactorValue..death.from.disease.,FactorValue..high.risk.,FactorValue..inss.stage.,FactorValue..progression.
ID,,,,,,
NB498,female,530,NaN,NaN,NaN,NaN
NB497,female,379,0.0,0.0,1,0.0
NB496,male,132,NaN,NaN,NaN,NaN
NB495,male,163,0.0,0.0,1,0.0
NB494,male,56,NaN,NaN,NaN,NaN


####  Divide into training and external testing

As you might have already noticed, we removed some of the factor values for some of the patient **ID**s.
Every row, where this information is missing indicate a real validation entry. We can use this information and create two separate DataFrames, one for training, one for the validation (testing). 

The task is to predict the missing values, either with the RNASeq or the Microarray data, or potentially both?



In [10]:
df_patient_info_train  = df_patient_info[df_patient_info['FactorValue..death.from.disease.'].notna()]
df_patient_info_test   = df_patient_info[df_patient_info['FactorValue..death.from.disease.'].isna()]



In [51]:
df_patient_info_train.head(20)

FactorValues,FactorValue..Sex.,FactorValue..age.at.diagnosis.,FactorValue..death.from.disease.,FactorValue..high.risk.,FactorValue..inss.stage.,FactorValue..progression.
ID,,,,,,
NB497,female,379,0.0,0.0,1,0.0
NB495,male,163,0.0,0.0,1,0.0
NB493,male,190,0.0,0.0,1,0.0
NB491,male,2326,0.0,1.0,4,1.0
NB489,female,865,0.0,1.0,4,0.0
NB487,male,162,0.0,1.0,4,0.0
NB485,female,712,0.0,1.0,3,0.0
NB483,female,259,0.0,0.0,1,1.0
NB481,male,2045,0.0,1.0,4,0.0


In [11]:
train_on_genes = df_patient_info_train.join(tp_gene_of_i, on="ID")
train_on_genes.head(10)

,FactorValue..Sex.,FactorValue..age.at.diagnosis.,FactorValue..death.from.disease.,FactorValue..high.risk.,FactorValue..inss.stage.,FactorValue..progression.,36700,35332,40594,813,...,44494,34963,42490,36855,42910,34175,44328,36747,43552,38595
ID,,,,,,,,,,,,,,,,,,,,,
NB497,female,379,0.0,0.0,1,0.0,8.4,6.62,11.84,3.28,...,14.36,6.69,11.52,11.12,7.78,9.05,8.26,12.67,13.06,9.42
NB495,male,163,0.0,0.0,1,0.0,9.1,6.06,11.55,2.41,...,14.22,6.75,11.54,10.5,7.45,8.96,9.1,12.84,13.62,8.95
NB493,male,190,0.0,0.0,1,0.0,8.44,4.86,11.65,2.58,...,14.39,5.93,11.21,10.96,7.24,8.42,8.3,12.62,12.76,8.74
NB491,male,2326,0.0,1.0,4,1.0,10.07,4.52,10.33,1.38,...,13.39,4.8,11.43,10.63,6.43,7.22,7.5,12.77,11.75,9.51
NB489,female,865,0.0,1.0,4,0.0,9.04,3.5,9.68,3.32,...,12.43,5,10.76,10.77,6.11,7.63,8.72,12,11.72,9.52
NB487,male,162,0.0,1.0,4,0.0,8.13,3.07,9.94,1.68,...,13.03,4.6,11.25,10.67,6.1,7.87,8.62,12.38,12.17,9.93
NB485,female,712,0.0,1.0,3,0.0,8.1,7.46,10.39,1.54,...,12.49,5.3,9.55,9.71,5.35,7.15,7.02,13.23,11.6,7.54
NB483,female,259,0.0,0.0,1,1.0,8.17,3.98,9.52,2.35,...,13.08,8.8,10.85,9.61,7.43,7.69,8.39,12.27,12.17,8.47
NB481,male,2045,0.0,1.0,4,0.0,8.39,4.91,10.81,3.26,...,12.5,4.69,10.02,8.99,5.66,6.88,7.51,13.88,11.92,10.53


In [12]:
df_patient_info_test.head()

FactorValues,FactorValue..Sex.,FactorValue..age.at.diagnosis.,FactorValue..death.from.disease.,FactorValue..high.risk.,FactorValue..inss.stage.,FactorValue..progression.
ID,,,,,,
NB498,female,530,NaN,NaN,NaN,NaN
NB496,male,132,NaN,NaN,NaN,NaN
NB494,male,56,NaN,NaN,NaN,NaN
NB492,male,947,NaN,NaN,NaN,NaN
NB490,female,1759,NaN,NaN,NaN,NaN


In [13]:
test_on_genes = df_patient_info_test.join(tp_gene_of_i, on="ID")
test_on_genes.head(10)

,FactorValue..Sex.,FactorValue..age.at.diagnosis.,FactorValue..death.from.disease.,FactorValue..high.risk.,FactorValue..inss.stage.,FactorValue..progression.,36700,35332,40594,813,...,44494,34963,42490,36855,42910,34175,44328,36747,43552,38595
ID,,,,,,,,,,,,,,,,,,,,,
NB498,female,530,NaN,NaN,NaN,NaN,8.5,7.09,10.88,3.81,...,13.47,9.71,10.97,10.48,7.42,9,7.66,11.97,11.87,8.6
NB496,male,132,NaN,NaN,NaN,NaN,8.84,4.15,11.4,3.64,...,14.08,6.2,11.42,10.68,8.15,8.61,8.54,12.46,12.13,9.05
NB494,male,56,NaN,NaN,NaN,NaN,8.72,5.28,10.91,3.79,...,13.89,5,10.82,10.09,7.42,8.75,8.43,12.88,13.44,8.7
NB492,male,947,NaN,NaN,NaN,NaN,8.59,4.9,9.27,2.07,...,12.62,5.43,10.98,10.23,5.46,7.83,8.8,12.79,11.7,9.32
NB490,female,1759,NaN,NaN,NaN,NaN,7.95,5.43,8.6,0.58,...,12.6,3.55,9.21,10.14,6.39,8.36,7.34,12.82,11.09,9.16
NB488,female,212,NaN,NaN,NaN,NaN,8.32,4.43,9.57,1.58,...,12.7,3.99,9.32,10.13,4.72,7.84,5.09,11.95,11.14,8.35
NB486,male,478,NaN,NaN,NaN,NaN,9.19,5.28,11.12,2.61,...,14.11,5.08,11.26,10.58,6.72,6.94,8.41,13.95,13.38,9.14
NB484,female,1546,NaN,NaN,NaN,NaN,7.74,3.78,9.44,2,...,11.95,3.64,9.19,8.99,4.6,6.08,6.97,12.56,11.2,7.51
NB482,male,465,NaN,NaN,NaN,NaN,9.36,7.4,11.35,2.68,...,14.19,4.71,11.23,10.62,7.23,9.12,8.32,13.26,13.08,9.03


## Analysis

From here on, you will need to use your skills ...

### Note:
To be clear, there are **multiple** target features/attributes to predict. Say you want to build a model predicting **death from disease** of a patient, your target variable is ```'FactorValue..death.from.disease.'``` and the corresponding target vector <em>y</em> would be as follows:

```python
y_train = df_patient_info_train['FactorValue..death.from.disease.'].astype(int)
```
Taking the other data into account (RNASeq or microarray) as ``` X_train``` (you will have to preprocess and split this information yourselves), you could for example build a random forest model:

```python
from sklearn.ensemble import RandomForestClassifier
random_f_model_death = random_f_model = RandomForestClassifier() 
random_f_model_death.fit(X_train,y_train)
```
and predict ```y_test``` using ```X_test```. 

Obviously, you want to avoid any overfitting and might want to use appropriate validation approaches. 

Once you have your model and the prediction for the test data, you should be able to fill the ```'FactorValue..death.from.disease.'``` column in the test set. 

For the submission, please also include the confidence/probability/score for each of the prediction (assume ```1``` to be the value for the positive class). This only applies to the factors:  **high risk**, **progression** and **death from disease** . 

The other two factors need to be treated differently. 

More information on how to submit the results will follow. 

Using this very rough outline, you should be able to predict all factors in the test data. 











In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [15]:
imputer = KNNImputer(n_neighbors=5, weights='uniform')

In [17]:
imputer.fit(train_on_genes, test_on_genes)

ValueError: could not convert string to float: 'female'